In [1]:
from RuNNE import RuNNEBuilder

builder = RuNNEBuilder()
builder.download_and_prepare()
dataset = builder.as_dataset()
df = dataset['train'].to_pandas().drop(columns='id')
df

,text,entities
0,Ким Чен Нама убили с помощью запрещённого химо...,"[0 12 PERSON, 56 72 ORGANIZATION, 64 72 COUNTR..."
1,Смена портретов на долларах\nГарриет Табмен\nК...,"[19 27 MONEY, 28 42 PERSON, 52 67 MONEY, 68 77..."
2,Новым генсеком ООН станет португалец Гутерреш\...,"[6 14 PROFESSION, 15 18 ORGANIZATION, 26 36 NA..."
3,Вахту принял\n\nУоррен Баффет назвал своего по...,"[14 27 PERSON, 199 209 DATE, 232 260 PROFESSIO..."
4,В Японии скончался старейший житель Земли — Дз...,"[2 8 COUNTRY, 79 96 DATE, 116 122 COUNTRY, 134..."
...,...,...
456,Электрик руководит социал-демократами Дюссельд...,"[0 8 PROFESSION, 68 72 ORGANIZATION, 73 101 TI..."
457,Мужчина женился на тёще\n\nРумынский мужчина у...,"[25 34 NATIONALITY, 129 138 PERSON, 154 161 CI..."
458,Названы лауреаты премии «World Press Photo»\nФ...,"[61 67 COUNTRY, 68 78 PERSON, 258 286 ORGANIZA..."
459,Первое поражение Серены Уильямс в финале Austr...,"[0 6 ORDINAL, 17 31 PERSON, 41 56 EVENT, 57 71..."


In [2]:
# load the model
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
train = dataset['train']
test = dataset['test']
dev = dataset['dev']

# preprocess the dataset
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['text']
    _targets = examples['entities']
    targets = []
    for t in _targets:
        targets.append(' '.join(t))
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    #model_inputs.drop
    return model_inputs

In [4]:
train_processed = train.map(preprocess_function, batched=True)
test_processed = test.map(preprocess_function, batched=True)
#dev_processed = dev.map(preprocess_function, batched=True)

Map:   0%|          | 0/461 [00:00<?, ? examples/s]

C:\Users\Demid\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [5]:
train_processed

Dataset({
    features: ['id', 'text', 'entities', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 461
})

In [28]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import pandas as pd

batch_size = 1
args = Seq2SeqTrainingArguments(
    "T5-Small-RuNNE",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True, # set to True if you have CUDA, False if CUDA is not available
    generation_max_length=96,
    
)

# set up the data collator to pad the inputs and labels
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# set up the metrics for the training process.
from datasets import load_metric

metric = load_metric("sacrebleu") # using the metric from the example

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# set up the trainer itself
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_processed,
    eval_dataset=test_processed,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# do the thing. That would take some time.
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.048456,0.028600,95.000000
2,2.108900,2.015671,0.031600,95.000000


In [25]:
trainer.save_model('T5Small-RuNNE-64')

In [26]:
tokenizer_trained = T5Tokenizer.from_pretrained('T5Small-RuNNE-64')
model_trained = T5ForConditionalGeneration.from_pretrained('T5Small-RuNNE-64')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
# test if it is working on a part of test dataset
from tqdm.notebook import trange
input = []
result = [] 
target = []
N = 10

for i in trange(N):
    input_ids = tokenizer_trained(test['text'][i], return_tensors="pt").input_ids
    outputs = model_trained.generate(input_ids, max_new_tokens = 128)
    input.append(test['text'][i])
    result.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    target.append(test['entities'][i])

pd.DataFrame.from_dict({'input': input, 'result': result, 'target': target}).head(N)

  0%|          | 0/10 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (913 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 